Model Traning 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

In [2]:
df = pd.read_csv(r'\Users\alexc\Heart_failure_predictor\notebook\heart_failure.csv')

In [3]:
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [4]:
print("Categories in 'gender' variable:     ",end=" " )
print(df['ChestPainType'].unique())

print("Categories in 'race_ethnicity' variable:  ",end=" ")
print(df['RestingECG'].unique())

print("Categories in'parental level of education' variable:",end=" " )
print(df['ExerciseAngina'].unique())

print("Categories in 'lunch' variable:     ",end=" " )
print(df['ST_Slope'].unique())

Categories in 'gender' variable:      ['ATA' 'NAP' 'ASY' 'TA']
Categories in 'race_ethnicity' variable:   ['Normal' 'ST' 'LVH']
Categories in'parental level of education' variable: ['N' 'Y']
Categories in 'lunch' variable:      ['Up' 'Flat' 'Down']


In [19]:
X = df.drop(columns=['HeartDisease'])
y = df['HeartDisease']

In [20]:
num_features = df.select_dtypes(exclude="object").columns
cat_features = df.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
         ("StandardScaler", numeric_transformer, num_features),        
    ]
)

X = preprocessor.fit_transform(df)
X.view()

array([[ 0.        ,  1.        ,  0.        , ...,  1.38292822,
        -0.83243239, -1.11311472],
       [ 1.        ,  0.        ,  0.        , ...,  0.75415714,
         0.10566353,  0.89837999],
       [ 0.        ,  1.        ,  0.        , ..., -1.52513802,
        -0.83243239, -1.11311472],
       ...,
       [ 0.        ,  1.        ,  1.        , ..., -0.85706875,
         0.29328271,  0.89837999],
       [ 1.        ,  0.        ,  0.        , ...,  1.4615246 ,
        -0.83243239,  0.89837999],
       [ 0.        ,  1.        ,  0.        , ...,  1.42222641,
        -0.83243239, -1.11311472]])

In [21]:
X.shape

(918, 21)

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape

((734, 21), (184, 21))

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

class LogisticRegressionFactory:
    @staticmethod
    def create_model(lr):
        # Return an instance of LogisticRegression with given learning rate (lr)
        return LogisticRegression(lr)

class LogisticRegression:
    def __init__(self, lr):
        self.lr = lr
        self.w = None

    def fit(self, P, y):
        # Initializing weights and gradient descent
        self.w = np.zeros(P.shape[1])
        self.w[0] = 1
        grad_norm = 1

        while grad_norm > 0.002:
            # Computing the gradient descent
            grad_dis = np.matmul(P.transpose(), sigmoid(np.matmul(self.w, P.transpose())) - y)

            # Updating the weights using gradient descent
            self.w = self.w - self.lr * grad_dis

            # Compute the norm of the gradient descent
            grad_norm = np.linalg.norm(grad_dis)

    def predict(self, p):
        return sigmoid(np.sum(self.w * p))

# Rest of the code remains unchanged

accuracy_list = []

# Number of runs to perform
tests = 10

scaler = StandardScaler()

# Reading the data setting to numpy array and splitit into 90% training samples and 10% test samples
data = pd.read_csv('exams.csv').to_numpy()

for t in range(tests):
    train, test = train_test_split(data, test_size=0.1)

    # Scaling the data and splitting to data and labels
    train, train_labels = train[:, :2], train[:, 2]
    test, test_labels = test[:, :2], test[:, 2]

    train = scaler.fit_transform(train)
    test = scaler.fit_transform(test)

    lr = 0.05

    model = LogisticRegressionFactory.create_model(lr)
    model.fit(train, train_labels)

    count = 0

    # Computing the predictions
    for label, p in zip(test_labels, test):
        prediction = model.predict(p)
        if prediction >= 0.5:
            predicted_label = 1
        else:
            predicted_label = 0
        if predicted_label == label:
            count += 1

    # Calculating the accuracy
    accuracy = count / len(test_labels)
    accuracy_list.append(accuracy)
    data_labels = data[:, 2]